In [1]:
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb

In [3]:
max_features=10000
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max_features)

In [4]:
#padding the dataset
max_len=500
x_train=sequence.pad_sequences(x_train,maxlen=max_len)
x_test=sequence.pad_sequences(x_test,maxlen=max_len)

In [5]:
x_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [6]:
x_test

array([[   0,    0,    0, ...,   14,    6,  717],
       [   0,    0,    0, ...,  125,    4, 3077],
       [  33,    6,   58, ...,    9,   57,  975],
       ...,
       [   0,    0,    0, ...,   21,  846, 5518],
       [   0,    0,    0, ..., 2302,    7,  470],
       [   0,    0,    0, ...,   34, 2005, 2643]], dtype=int32)

In [7]:
model=Sequential()
model.add(Embedding(max_features,128,input_length=max_len))
model.add(SimpleRNN(128,activation='relu')) # hidden layer we are using simple rnn as it has feedback layer taht ann doesn't have
model.add(Dense(1,activation='sigmoid')) # output layer

d:\deep learning\RNN\venv\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
model.build(input_shape=(None, max_len))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
#create instace for earlystopping
from tensorflow.keras.callbacks import EarlyStopping
EarlyStopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True) 

In [10]:
EarlyStopping

In [11]:
#train the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history=model.fit(x_train,y_train,epochs=10,batch_size=32,validation_split=0.2,callbacks=[EarlyStopping])

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 123s 191ms/step - accuracy: 0.5440 - loss: 21339090.0000 - val_accuracy: 0.6056 - val_loss: 0.6703
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 134s 215ms/step - accuracy: 0.6638 - loss: 588.1630 - val_accuracy: 0.6368 - val_loss: 0.6495
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 147s 235ms/step - accuracy: 0.6646 - loss: 74508.2500 - val_accuracy: 0.6432 - val_loss: 0.7557
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 144s 231ms/step - accuracy: 0.6983 - loss: 0.6314 - val_accuracy: 0.6578 - val_loss: 0.6852
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 229ms/step - accuracy: 0.7478 - loss: 0.5255 - val_accuracy: 0.6772 - val_loss: 0.6568
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 145s 231ms/step - accuracy: 0.7876 - loss: 0.4589 - val_accuracy: 0.6894 - val_loss: 0.6401
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3688s 6s/step - accuracy: 0.8199 - loss: 0.4060 - val_accuracy: 0.6998 - val_loss: 0.6308
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 168s 186ms/step - accurac

In [12]:
#load model 
model.save('imdb.h5')